## Linear Regression
Coder: Gomathi Sengodan
Date: Oct 2nd 2019

In [244]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from numpy import array
from numpy import argmax
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn import datasets, linear_model, metrics 
#from sklearn.cross_validation import cross_val_score
import pickle
import statsmodels.api as sm

In [262]:
X = pd.read_csv("playfinal.csv")
print(X.shape)
numcols = X.select_dtypes(include =['int64','float64'])
print(numcols.columns)
X = X[numcols.columns]
Y1 = X['raidrating']
Y2 = X['tacklerating']
X.drop(['raidrating','tacklerating','year'],axis=1,inplace=True)


(408, 25)
Index(['Unnamed: 0', 'Matchplayed', 'tpoints', 'mostpoints', 'notout',
       'traids', 'successraidspercent', 'noofsuperraids', 'super10',
       'traidpoints', 'avgraidpointspermatch', 'supertackle', 'high5s',
       'totaltacklepoints', 'avgtackperm', 'totaltackle', 'totalstrike',
       'year', 'successtackles', 'successtacklepercent', 'successfulraid',
       'raidrating', 'tacklerating'],
      dtype='object')


# Loading season 7 data for prediction

In [264]:

test = pd.read_csv("predictfinal.csv") 
playname=test.name
numcols = test.select_dtypes(include =['int64','float64'])
test = test[numcols.columns]
print(test.shape)
test = test[test.Matchplayed > 10] # teams qualified
print(test.shape)
Y1test = test['raidrating']
Y2test = test['tacklerating']
test.drop(['raidrating','tacklerating'],axis=1,inplace=True)

(178, 22)
(100, 22)


# Top players based on Successful raid percent

In [265]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y1, test_size=0.15, random_state=42)
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#X_train[numcols.columns] = scaler.fit_transform(X_train[numcols.columns])
#X_test[numcols.columns] = scaler.transform(X_test[numcols.columns])


X_train = X
y_train = Y1
X_test = test
y_test = Y1test
 

In [266]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, 5)             # running RFE
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

[('Unnamed: 0', False, 16),
 ('Matchplayed', False, 11),
 ('tpoints', False, 2),
 ('mostpoints', False, 6),
 ('notout', False, 14),
 ('traids', False, 15),
 ('successraidspercent', False, 8),
 ('noofsuperraids', False, 10),
 ('super10', False, 5),
 ('traidpoints', True, 1),
 ('avgraidpointspermatch', True, 1),
 ('supertackle', False, 4),
 ('high5s', False, 7),
 ('totaltacklepoints', False, 3),
 ('avgtackperm', True, 1),
 ('totaltackle', False, 9),
 ('totalstrike', False, 13),
 ('successtackles', True, 1),
 ('successtacklepercent', False, 12),
 ('successfulraid', True, 1)]

In [267]:
# Build model using stats
colrfe = X_train.columns[rfe.support_]
X_train_rfe = sm.add_constant(X_train[colrfe])
lm = sm.OLS(y_train,X_train_rfe).fit() 
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:             raidrating   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     51.80
Date:                Wed, 02 Oct 2019   Prob (F-statistic):           2.08e-41
Time:                        22:16:58   Log-Likelihood:                -1900.0
No. Observations:                 408   AIC:                             3812.
Df Residuals:                     402   BIC:                             3836.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    61.67

G:\samsung\pyth\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [268]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_rfe
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
5,successfulraid,1.848987e+08
1,traidpoints,1.848941e+08
2,avgraidpointspermatch,1.106000e+01
3,avgtackperm,6.050000e+00
4,successtackles,5.810000e+00
0,const,3.490000e+00


In [269]:
# Build model using stats on test data 
print(X_test[colrfe].head())
X_test_rfe = sm.add_constant(X_test[colrfe])
y_res = lm.predict(X_test_rfe)
df = pd.DataFrame({'name':playname,'rating':y_res}) 
 

   traidpoints  avgraidpointspermatch  avgtackperm  successtackles  \
0         99.0                   5.50         0.11             2.0   
1        205.0                  10.25         0.00             0.0   
5         33.0                   2.75         0.08             1.0   
8          0.0                   0.00         2.68            51.0   
9          0.0                   0.00         1.66            30.0   

   successfulraid  
0         98.9925  
1        204.9828  
5         32.9960  
8          0.0000  
9          0.0000  


G:\samsung\pyth\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [270]:
df.sort_values('rating',ascending=True)

,name,rating
99,pardeepnar,-25.860695
22,pawans,-24.771506
40,naveenkumardd,-21.270765
73,vikashk,-14.658804
147,siddharth,-7.785477
1,maninder,-4.913853
117,pankajmohit,6.990375
155,abhihmumb,8.086005
83,deepakhooda,15.258976
171,shrinkant,16.168160


# Top players based on successful tackle percent

In [271]:

X = pd.read_csv("playfinal.csv")
print(X.shape)
numcols = X.select_dtypes(include =['int64','float64'])
print(numcols.columns)
X = X[numcols.columns]
Y1 = X['raidrating']
Y2 = X['tacklerating']
X.drop(['raidrating','tacklerating','year'],axis=1,inplace=True)

test = pd.read_csv("predictfinal.csv") 
playname=test.name
numcols = test.select_dtypes(include =['int64','float64'])
test = test[numcols.columns]
test = test[test.Matchplayed > 10] # teams qualified
Y1test = test['raidrating']
Y2test = test['tacklerating']
test.drop(['raidrating','tacklerating'],axis=1,inplace=True)

X_train = X
y_train = Y2
X_test = test
y_test = Y2test
 # Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, 10)             # running RFE
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

(408, 25)
Index(['Unnamed: 0', 'Matchplayed', 'tpoints', 'mostpoints', 'notout',
       'traids', 'successraidspercent', 'noofsuperraids', 'super10',
       'traidpoints', 'avgraidpointspermatch', 'supertackle', 'high5s',
       'totaltacklepoints', 'avgtackperm', 'totaltackle', 'totalstrike',
       'year', 'successtackles', 'successtacklepercent', 'successfulraid',
       'raidrating', 'tacklerating'],
      dtype='object')


[('Unnamed: 0', False, 11),
 ('Matchplayed', False, 7),
 ('tpoints', True, 1),
 ('mostpoints', False, 6),
 ('notout', False, 9),
 ('traids', False, 8),
 ('successraidspercent', False, 10),
 ('noofsuperraids', True, 1),
 ('super10', False, 2),
 ('traidpoints', True, 1),
 ('avgraidpointspermatch', True, 1),
 ('supertackle', True, 1),
 ('high5s', False, 3),
 ('totaltacklepoints', True, 1),
 ('avgtackperm', True, 1),
 ('totaltackle', False, 4),
 ('totalstrike', False, 5),
 ('successtackles', True, 1),
 ('successtacklepercent', True, 1),
 ('successfulraid', True, 1)]

In [272]:
# Build model using stats
colrfe = X_train.columns[rfe.support_]
X_train_rfe = sm.add_constant(X_train[colrfe])
lm = sm.OLS(y_train,X_train_rfe).fit() 
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:           tacklerating   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     61.17
Date:                Wed, 02 Oct 2019   Prob (F-statistic):           1.04e-64
Time:                        22:17:51   Log-Likelihood:                -1838.2
No. Observations:                 408   AIC:                             3694.
Df Residuals:                     399   BIC:                             3730.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    78.38

G:\samsung\pyth\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [273]:
# Build model using stats on test data 
print(X_test[colrfe].head())
X_test_rfe = sm.add_constant(X_test[colrfe])
y_res = lm.predict(X_test_rfe)
df = pd.DataFrame({'name':playname,'tacklerating':y_res}) 
 

   tpoints  noofsuperraids  traidpoints  avgraidpointspermatch  supertackle  \
0      101             4.0         99.0                   5.50          0.0   
1      205             6.0        205.0                  10.25          0.0   
5       34             2.0         33.0                   2.75          0.0   
8       56             0.0          0.0                   0.00          5.0   
9       31             0.0          0.0                   0.00          1.0   

   totaltacklepoints  avgtackperm  successtackles  successtacklepercent  \
0                2.0         0.11             2.0             14.285714   
1                0.0         0.00             0.0              0.000000   
5                1.0         0.08             1.0             25.000000   
8               56.0         2.68            51.0             51.000000   
9               31.0         1.66            30.0             37.500000   

   successfulraid  
0         98.9925  
1        204.9828  
5         32.9

In [274]:
df.sort_values('tacklerating',ascending=True)

,name,tacklerating
58,sonugf,-0.303446
176,sumitsin,6.332569
154,fazel,8.523174
8,Baldev,14.276828
151,bharad,14.481338
56,moregb,14.915462
166,niteshk,14.987695
77,sunilhs,17.623889
165,sandeepmumba,17.986353
91,amitjaipur,18.184704


# Top players based on successful raids

In [276]:
X = pd.read_csv("playratedwithsuccesscount.csv")
print(X.shape)
numcols = X.select_dtypes(include =['int64','float64'])
print(numcols.columns)
X = X[numcols.columns]
Y1 = X['raidrating']
Y2 = X['tacklerating']
X.drop(['raidrating','tacklerating','year'],axis=1,inplace=True)
test = pd.read_csv("predictfinal.csv") 
playname=test.name
numcols = test.select_dtypes(include =['int64','float64'])
test = test[numcols.columns]
Y1test = test['raidrating']
Y2test = test['tacklerating']
test.drop(['raidrating','tacklerating'],axis=1,inplace=True)


(408, 25)
Index(['Unnamed: 0', 'Matchplayed', 'tpoints', 'mostpoints', 'notout',
       'traids', 'successraidspercent', 'noofsuperraids', 'super10',
       'traidpoints', 'avgraidpointspermatch', 'supertackle', 'high5s',
       'totaltacklepoints', 'avgtackperm', 'totaltackle', 'totalstrike',
       'year', 'successtackles', 'successtacklepercent', 'successfulraid',
       'raidrating', 'tacklerating'],
      dtype='object')


In [277]:
X_train = X
y_train = Y1
X_test = test
y_test = Y1test
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, 5)             # running RFE
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

# Build model using stats
colrfe = X_train.columns[rfe.support_]
X_train_rfe = sm.add_constant(X_train[colrfe])
lm = sm.OLS(y_train,X_train_rfe).fit() 
print(lm.summary())
# Build model using stats on test data 

X_test_rfe = sm.add_constant(X_test[colrfe])
y_res = lm.predict(X_test_rfe)
df = pd.DataFrame({'name':playname,'rating':y_res}) 
df.sort_values('rating',ascending=True)

                            OLS Regression Results                            
Dep. Variable:             raidrating   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     62.74
Date:                Wed, 02 Oct 2019   Prob (F-statistic):           3.38e-41
Time:                        22:18:55   Log-Likelihood:                -1902.7
No. Observations:                 408   AIC:                             3815.
Df Residuals:                     403   BIC:                             3835.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                53.4604      2.10

G:\samsung\pyth\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,name,rating
40,naveenkumardd,-33.417830
99,pardeepnar,-31.470758
22,pawans,-23.223264
1,maninder,-16.681087
147,siddharth,-0.296914
73,vikashk,3.210506
83,deepakhooda,4.789882
171,shrinkant,16.003906
155,abhihmumb,16.604403
131,rahulchau,18.053201


# Top players based on successful tackles

In [278]:
X_train = X
y_train = Y2
X_test = test
y_test = Y2test
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, 8)             # running RFE
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

# Build model using stats
colrfe = X_train.columns[rfe.support_]
X_train_rfe = sm.add_constant(X_train[colrfe])
lm = sm.OLS(y_train,X_train_rfe).fit() 
print(lm.summary())
# Build model using stats on test data 

X_test_rfe = sm.add_constant(X_test[colrfe])
y_res = lm.predict(X_test_rfe)
df = pd.DataFrame({'name':playname,'rating':y_res}) 
df.sort_values('rating',ascending=True)

                            OLS Regression Results                            
Dep. Variable:           tacklerating   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.479
Method:                 Least Squares   F-statistic:                     63.49
Date:                Wed, 02 Oct 2019   Prob (F-statistic):           3.46e-55
Time:                        22:20:27   Log-Likelihood:                -1865.2
No. Observations:                 408   AIC:                             3744.
Df Residuals:                     401   BIC:                             3773.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    69.52

G:\samsung\pyth\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,name,rating
93,sandeepdhull,-9.401032
151,bharad,-8.452925
176,sumitsin,-7.973672
154,fazel,-3.633319
8,Baldev,1.144783
28,mahenders,5.278432
166,niteshk,5.659976
98,vishaljai,6.240208
112,surjeetpune,7.059319
77,sunilhs,7.090548


In [279]:
numcols

,Unnamed: 0,Matchplayed,tpoints,mostpoints,notout,traids,successraidspercent,noofsuperraids,super10,traidpoints,...,high5s,totaltacklepoints,avgtackperm,totaltackle,totalstrike,successtackles,successtacklepercent,successfulraid,raidrating,tacklerating
0,0,18,101,12.0,79.18,197.0,50.25,4.0,3.0,99.0,...,0.0,2.0,0.11,14.0,14.28,2.0,14.285714,98.9925,0,0
1,2,20,205,19.0,82.07,318.0,64.46,6.0,10.0,205.0,...,0.0,0.0,0.00,7.0,0.00,0.0,0.000000,204.9828,0,0
2,4,3,1,1.0,25.00,4.0,0.00,0.0,0.0,0.0,...,0.0,1.0,0.33,4.0,25.00,1.0,25.000000,0.0000,0,0
3,6,1,3,3.0,50.00,2.0,150.00,0.0,0.0,3.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,3.0000,0,0
4,8,2,2,2.0,57.14,7.0,14.28,0.0,0.0,1.0,...,0.0,1.0,0.50,1.0,100.00,1.0,100.000000,0.9996,0,0
5,10,12,34,8.0,82.19,73.0,45.20,2.0,0.0,33.0,...,0.0,1.0,0.08,4.0,25.00,1.0,25.000000,32.9960,0,0
6,12,3,1,1.0,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,1.0,0.33,8.0,12.50,1.0,12.500000,0.0000,0,0
7,14,4,0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,1.0,0.00,0.0,0.000000,0.0000,0,0
8,16,19,56,7.0,80.00,5.0,0.00,0.0,0.0,0.0,...,6.0,56.0,2.68,100.0,56.00,51.0,51.000000,0.0000,0,0
9,18,18,31,4.0,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,31.0,1.66,80.0,38.75,30.0,37.500000,0.0000,0,0


## Results

For Linear Regression, top 3 ratings of players are :
Successful raid percent -
Successful raids
Successful tackle percent
Successful tackle


